<a href="https://www.kaggle.com/code/hari141v/semantic-search-with-vectors-elasticsearch?scriptVersionId=187558793" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Import necessary libraries and packages

In [1]:
pip install elasticsearch -qq 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence_transformers==2.7.0 -qq

Note: you may need to restart the kernel to use updated packages.


In [3]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
!tar -xzf elasticsearch-8.14.0-linux-x86_64.tar.gz
!chown -R root:root elasticsearch-8.14.0
!adduser --disabled-password --gecos '' elasticsearch_user
!chown -R elasticsearch_user:elasticsearch_user elasticsearch-8.14.0

--2024-07-09 17:51:49--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.14.0-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588482330 (561M) [application/x-gzip]
Saving to: 'elasticsearch-8.14.0-linux-x86_64.tar.gz'

elasticsearch-8.14. 100%[===================>] 561.22M  27.3MB/s    in 21s     

2024-07-09 17:52:11 (26.8 MB/s) - 'elasticsearch-8.14.0-linux-x86_64.tar.gz' saved [588482330/588482330]

Adding user `elasticsearch_user' ...
Adding new group `elasticsearch_user' (1000) ...
Adding new user `elasticsearch_user' (1001) with group `elasticsearch_user' ...
Creating home directory `/home/elasticsearch_user' ...
Copying files from `/etc/skel' ...


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io

import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json
import elasticsearch
from elasticsearch import Elasticsearch

### Elasticsearch setup

In [5]:
# Run elasticsearch
command = "sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(20)

In [6]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

root         126      33  0 17:52 ?        00:00:00 /bin/sh -c sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
root         127     126  0 17:52 ?        00:00:00 sudo -u elasticsearch_user ./elasticsearch-8.14.0/bin/elasticsearch -E xpack.security.enabled=false -E thread_pool.search.size=1 -E thread_pool.write.size=1
elastic+     128     127 32 17:52 ?        00:00:07 /kaggle/working/elasticsearch-8.14.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch-8.14.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/kaggle/working/elasticsearch-8.14.0 -Des.path.conf=/kaggle/working/elasticsearch-8.14.0/config -Des.distribution.type=tar -cp /kaggle/working/elasticsearch-8.14.0/lib/*:/kaggle/working/elasticsearch-8.14.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher -E xpack.security.enabled=false -E thread_pool.se

In [7]:
es_client = Elasticsearch('http://localhost:9200/', request_timeout=60) 

In [9]:
!curl localhost:9200

{
  "name" : "e71ce9a54156",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "tDZ3MBLVTyqluqZR6KFuTw",
  "version" : {
    "number" : "8.14.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "8d96bbe3bf5fed931f3119733895458eab75dca9",
    "build_date" : "2024-06-03T10:05:49.073003402Z",
    "build_snapshot" : false,
    "lucene_version" : "9.10.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### Get json file from course repo

In [10]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Creating embedding using sentence transformer

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Let's embed a sentence and see the dimension

In [13]:
len(model.encode("This is a simple sentence",show_progress_bar=False))

384

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Let's create embeding for text

In [15]:
len(documents)

948

In [16]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"],show_progress_bar=False,normalize_embeddings=True).tolist()
    operations.append(doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [17]:
for k,v in operations[1].items():
    print(f"{k}: {len(v)}")

text: 62
section: 32
question: 52
course: 25
text_vector: 384


### Create Mappings and Index for elastic search

In [18]:
model_dimension=384

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": model_dimension, "index": True, "similarity": "cosine"},
        }
    }
}

### Create index

In [19]:
index_name = "course-questions"

# delete the index if already exist
es_client.indices.delete(index=index_name, ignore_unavailable=True)

#Create index
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Add documents into index

In [20]:
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/948 [00:00<?, ?it/s]

###  Create query and embed it

In [39]:
search_term = "Windows or mac?"
vector_search_term = model.encode(search_term,show_progress_bar=False)

In [40]:
vector_search_term.shape

(384,)

### Search

In [23]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [24]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'V2WlmJABucq0u_AhGHWB',
  '_score': 0.7112181,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '8mWlmJABucq0u_AhL3bc',
  '_score': 0.6635225,
  '_source': {'text': 'For the Machine Learning part, all you need is a working laptop with an internet connection. The Deep Learning part is more resource intensive, but for that you can use a cloud (we use Saturn cloud but can be anything else).\n(Rileen Sinha; based on response by Alexey on Slack)',
   'section': 'General course-related questions',
   'question': "Any particular hardware requirements for the course or everything is mostly cloud? TIA! Couldn't really find this in the FAQ.",
   'course': 'machine

### Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [32]:
set([v for i in operations for k,v in i.items() if k=='section'])

{'+-General course questions',
 '1. Introduction to Machine Learning',
 '10. Kubernetes and TensorFlow Serving',
 '11. KServe',
 '2. Machine Learning for Regression',
 '3. Machine Learning for Classification',
 '4. Evaluation Metrics for Classification',
 '5. Deploying Machine Learning Models',
 '6. Decision Trees and Ensemble Learning',
 '8. Neural Networks and Deep Learning',
 '9. Serverless Deep Learning',
 'Course Management Form for Homeworks',
 'General course-related questions',
 'Miscellaneous',
 'Module 1: Docker and Terraform',
 'Module 1: Introduction',
 'Module 2: Experiment tracking',
 'Module 2: Workflow Orchestration',
 'Module 3: Data Warehousing',
 'Module 3: Orchestration',
 'Module 4: Deployment',
 'Module 4: analytics engineering with dbt',
 'Module 5: Monitoring',
 'Module 5: pyspark',
 'Module 6: Best practices',
 'Module 6: streaming with kafka',
 'Project',
 'Projects (Midterm and Capstone)',
 'Triggers in Mage via CLI',
 'Workshop 1 - dlthub',
 'Workshop 2 - Ri

In [34]:
set([v for i in operations for k,v in i.items() if k=='course'])

{'data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp'}

In [109]:
def semantic_search(search_query,add_match=None):
    vector_search_term = model.encode(search_query,show_progress_bar=False)
    
    knn_query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 10000
    }
    
    query = {
    "bool": {
        "must": [
            {"match": {"section": "machine-learning-zoomcamp"}}
        ]
    }}
    
    if add_match:
           query["bool"]["must"].append({"match": {"course": add_match}})
                

    response = es_client.search(
        index=index_name,
        query=query,
        knn=knn_query,
        size=5
    )
    result1 = {k: v for k,v in response["hits"]["hits"][0].items() if k != "_source"}
    result2 = ({k: v for k, v in response["hits"]["hits"][0]['_source'].items() if k != 'text_vector'})
    final_result = {**result1, **result2}
    return final_result

### Filtering general course related questions

In [105]:
semantic_search(search_query='windows or mac?')

{'_index': 'course-questions',
 '_id': 'K2WlmJABucq0u_AhMnfE',
 '_score': 3.748248,
 'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer',
 'section': '2. Machine Learning for Regression',
 'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
 'course': 'machine-learning-zoomcamp'}

### Let's try with additonal match filter

In [108]:
for filters in ['data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp']:
    print(semantic_search(search_query='windows or mac?',add_match=filters))
    print("-"*100)

{'_index': 'course-questions', '_id': 'V2WlmJABucq0u_AhGHWB', '_score': 0.7112181, 'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully', 'section': 'General course-related questions', 'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?', 'course': 'data-engineering-zoomcamp'}
----------------------------------------------------------------------------------------------------
{'_index': 'course-questions', '_id': 'K2WlmJABucq0u_AhMnfE', '_score': 4.675399, 'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer', 'section': '2. Machine Learning for Regression', 'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?", 'course': 'machine-learning-zoomcamp